## 1. セットアップ

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from collections import Counter

sys.path.append('/workspace/atma_22_ca/')
from configs.config import *

# 日本語フォント設定
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print(f"Input dir: {DIR_INPUT}")
print(f"Train crops dir: {DIR_TRAIN_CROPS}")
print(f"Test crops dir: {DIR_CROPS}")

## 2. データ読み込み

In [ ]:
# メタデータ読み込み
df_train = pd.read_csv(FILE_TRAIN_META)
df_test = pd.read_csv(FILE_TEST_META)

print(f"訓練データ: {df_train.shape}")
print(f"テストデータ: {df_test.shape}")
print(f"\n訓練データ列: {df_train.columns.tolist()}")
print(f"テストデータ列: {df_test.columns.tolist()}")

In [ ]:
# 最初の数行を確認
print("=== 訓練データ ===")
display(df_train.head())

print("\n=== テストデータ ===")
display(df_test.head())

## 3. 基本統計

In [ ]:
# 訓練データの基本統計
print("=== 訓練データ基本統計 ===")
print(f"総サンプル数: {len(df_train):,}")
print(f"選手数: {df_train['label_id'].nunique()}")
print(f"Quarter数: {df_train['quarter'].nunique()}")
print(f"Session数: {df_train['session'].nunique()}")
print(f"画角: {df_train['angle'].unique()}")
print(f"\n選手ごとのサンプル数:")
print(df_train['label_id'].value_counts().sort_index())

In [ ]:
# テストデータの基本統計
print("=== テストデータ基本統計 ===")
print(f"総サンプル数: {len(df_test):,}")
print(f"Quarter数: {df_test['quarter'].nunique()}")
print(f"Session数: {df_test['session_no'].nunique()}")
print(f"画角: {df_test['angle'].unique()}")
print(f"\nrel_path列の最初の5件:")
print(df_test['rel_path'].head())

## 4. 選手分布の可視化

In [ ]:
# 選手ごとのサンプル数
plt.figure(figsize=(12, 5))
label_counts = df_train['label_id'].value_counts().sort_index()
plt.bar(label_counts.index, label_counts.values, color='skyblue', edgecolor='black')
plt.xlabel('Player ID', fontsize=12)
plt.ylabel('Sample Count', fontsize=12)
plt.title('Training Data: Sample Distribution per Player', fontsize=14)
plt.xticks(label_counts.index)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"最小サンプル数: {label_counts.min()}")
print(f"最大サンプル数: {label_counts.max()}")
print(f"平均サンプル数: {label_counts.mean():.1f}")

## 5. Quarter・Session分布

In [ ]:
# 訓練データのQuarter分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Quarter分布
quarter_counts_train = df_train['quarter'].value_counts().sort_index()
axes[0].bar(range(len(quarter_counts_train)), quarter_counts_train.values, color='lightcoral', edgecolor='black')
axes[0].set_xlabel('Quarter', fontsize=12)
axes[0].set_ylabel('Sample Count', fontsize=12)
axes[0].set_title('Training Data: Quarter Distribution', fontsize=14)
axes[0].set_xticks(range(len(quarter_counts_train)))
axes[0].set_xticklabels(quarter_counts_train.index, rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Session分布（訓練データは全てsession=0）
session_counts_train = df_train['session'].value_counts().sort_index()
axes[1].bar(session_counts_train.index, session_counts_train.values, color='lightgreen', edgecolor='black')
axes[1].set_xlabel('Session', fontsize=12)
axes[1].set_ylabel('Sample Count', fontsize=12)
axes[1].set_title('Training Data: Session Distribution', fontsize=14)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# テストデータのQuarter・Session分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Quarter分布
quarter_counts_test = df_test['quarter'].value_counts().sort_index()
axes[0].bar(range(len(quarter_counts_test)), quarter_counts_test.values, color='lightcoral', edgecolor='black')
axes[0].set_xlabel('Quarter', fontsize=12)
axes[0].set_ylabel('Sample Count', fontsize=12)
axes[0].set_title('Test Data: Quarter Distribution', fontsize=14)
axes[0].set_xticks(range(len(quarter_counts_test)))
axes[0].set_xticklabels(quarter_counts_test.index, rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Session分布（テストデータは複数sessionがある）
session_counts_test = df_test['session_no'].value_counts().sort_index()
axes[1].bar(session_counts_test.index, session_counts_test.values, color='lightgreen', edgecolor='black')
axes[1].set_xlabel('Session', fontsize=12)
axes[1].set_ylabel('Sample Count', fontsize=12)
axes[1].set_title('Test Data: Session Distribution', fontsize=14)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"テストデータのSession数: {df_test['session_no'].nunique()}")
print(f"各Sessionのサンプル数:\n{session_counts_test}")

In [ ]:
# uarter・Sessionごとの出場選手の分析

# 訓練データ: Quarterごとの選手分布
print("=== 訓練データ: Quarterごとの選手出場状況 ===")
quarter_label_matrix = df_train.groupby(['quarter', 'label_id']).size().unstack(fill_value=0)

# ヒートマップで可視化
plt.figure(figsize=(14, 10))
sns.heatmap(quarter_label_matrix.T, cmap='YlOrRd', annot=True, fmt='d', cbar_kws={'label': 'Sample Count'})
plt.xlabel('Quarter', fontsize=12)
plt.ylabel('Player ID', fontsize=12)
plt.title('Training Data: Player Appearance per Quarter', fontsize=14)
plt.tight_layout()
plt.show()

# テストデータ: Quarter・Sessionの組み合わせ確認
print("\n=== テストデータ: Quarter × Session の組み合わせ ===")
test_quarter_session = df_test.groupby(['quarter', 'session_no']).size().reset_index(name='count')
print(test_quarter_session.head(20))
print(f"\n総組み合わせ数: {len(test_quarter_session)}")

## 6. 画角（angle）分布

In [ ]:
# 訓練データとテストデータの画角分布
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 訓練データ
angle_counts_train = df_train['angle'].value_counts()
axes[0].bar(angle_counts_train.index, angle_counts_train.values, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Angle', fontsize=12)
axes[0].set_ylabel('Sample Count', fontsize=12)
axes[0].set_title('Training Data: Angle Distribution', fontsize=14)
axes[0].grid(axis='y', alpha=0.3)

# テストデータ
angle_counts_test = df_test['angle'].value_counts()
axes[1].bar(angle_counts_test.index, angle_counts_test.values, color='orange', edgecolor='black')
axes[1].set_xlabel('Angle', fontsize=12)
axes[1].set_ylabel('Sample Count', fontsize=12)
axes[1].set_title('Test Data: Angle Distribution', fontsize=14)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("訓練データ:")
print(angle_counts_train)
print("\nテストデータ:")
print(angle_counts_test)
print("\n⚠️ テストデータはside画角のみ（訓練データはside/topの両方）")

## 7. BBox サイズ分布

In [ ]:
# BBoxのサイズ（幅・高さ）分布
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 訓練データ
axes[0, 0].hist(df_train['w'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Width (pixels)', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Training Data: BBox Width Distribution', fontsize=14)
axes[0, 0].grid(axis='y', alpha=0.3)

axes[0, 1].hist(df_train['h'], bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Height (pixels)', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title('Training Data: BBox Height Distribution', fontsize=14)
axes[0, 1].grid(axis='y', alpha=0.3)

# テストデータ
axes[1, 0].hist(df_test['w'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[1, 0].set_xlabel('Width (pixels)', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].set_title('Test Data: BBox Width Distribution', fontsize=14)
axes[1, 0].grid(axis='y', alpha=0.3)

axes[1, 1].hist(df_test['h'], bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Height (pixels)', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title('Test Data: BBox Height Distribution', fontsize=14)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("訓練データ BBox統計:")
print(df_train[['w', 'h']].describe())
print("\nテストデータ BBox統計:")
print(df_test[['w', 'h']].describe())

## 8. 画像サンプル確認

In [ ]:
# 訓練データ: 事前クロップ済み画像を確認
train_crop_dir = Path(DIR_TRAIN_CROPS)

print(f"訓練データクロップ画像ディレクトリ: {train_crop_dir}")
print(f"ディレクトリ存在: {train_crop_dir.exists()}")

if train_crop_dir.exists():
    crop_files = list(train_crop_dir.glob('*.jpg'))
    print(f"クロップ画像数: {len(crop_files)}")
    
    # ランダムに選手ごとに1枚ずつ表示
    fig, axes = plt.subplots(2, 6, figsize=(18, 6))
    axes = axes.flatten()
    
    for label_id in range(11):  # 0-10の選手
        # 該当選手のサンプルを1つ取得
        sample = df_train[df_train['label_id'] == label_id].iloc[0]
        img_path = train_crop_dir / f"{sample.name}.jpg"
        
        if img_path.exists():
            img = cv2.imread(str(img_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            axes[label_id].imshow(img)
            axes[label_id].set_title(f'Player {label_id}', fontsize=10)
            axes[label_id].axis('off')
        else:
            axes[label_id].text(0.5, 0.5, 'Not Found', ha='center', va='center')
            axes[label_id].axis('off')
    
    axes[-1].axis('off')  # 12番目は空白
    
    plt.suptitle('Training Data: Sample Images per Player', fontsize=16)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 訓練データのクロップ画像が見つかりません")
    print("   preprocess_train_crops.ipynb を実行してください")

In [ ]:
# テストデータ: rel_path経由で画像を確認
test_crop_dir = Path(DIR_INPUT)

print(f"テストデータクロップ画像ベースディレクトリ: {test_crop_dir}")

# ランダムに12枚表示
fig, axes = plt.subplots(2, 6, figsize=(18, 6))
axes = axes.flatten()

sample_indices = np.random.choice(len(df_test), min(12, len(df_test)), replace=False)

for i, idx in enumerate(sample_indices):
    sample = df_test.iloc[idx]
    img_path = test_crop_dir / sample['rel_path']
    
    if img_path.exists():
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axes[i].imshow(img)
        axes[i].set_title(f'Test {idx}\n{sample["angle"]}', fontsize=9)
        axes[i].axis('off')
    else:
        axes[i].text(0.5, 0.5, 'Not Found', ha='center', va='center')
        axes[i].axis('off')

plt.suptitle('Test Data: Random Sample Images', fontsize=16)
plt.tight_layout()
plt.show()

## 9. CV戦略の検討

In [ ]:
# Group K-Fold用のグループ列を作成
df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)

print(f"グループ数: {df_train['group'].nunique()}")
print(f"平均サンプル数/グループ: {len(df_train) / df_train['group'].nunique():.1f}")
print(f"\nグループごとのサンプル数（上位10件）:")
print(df_train['group'].value_counts().reset_index(drop=False).sort_values(by='group'))

In [ ]:
# グループごとの選手分布を確認（リーク防止のため）
print("各グループに含まれる選手数:")
group_label_counts = df_train.groupby('group')['label_id'].nunique().sort_values(ascending=False)
print(group_label_counts.head(10))

# グループサイズの分布
plt.figure(figsize=(14, 5))
group_sizes = df_train['group'].value_counts().sort_values(ascending=False)
plt.bar(range(len(group_sizes)), group_sizes.values, color='steelblue', alpha=0.7)
plt.xlabel('Group Index (sorted by size)', fontsize=12)
plt.ylabel('Sample Count', fontsize=12)
plt.title('Group Size Distribution (for Group K-Fold)', fontsize=14)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 10. まとめ

### データ特徴
1. **訓練データ**: 全選手のサンプルあり、side/top両方の画角
2. **テストデータ**: side画角のみ、複数session
3. **BBoxサイズ**: 訓練/テストで分布が異なる可能性

### CV戦略
- **Group K-Fold**: `quarter_session`でグループ化
- **理由**: 同じシーンの画像を訓練/検証に分けないため（リーク防止）

### 注意点
- テストデータは**side画角のみ**なので、topで過学習しないよう注意
- sessionが0のみの訓練データと、複数sessionのテストデータで分布が異なる
- unknown選手（-1）の判定が重要